In [39]:
# Permet de tout executer au lancement du notebook + conserver le notebook actif pendant 2h
from IPython.display import Javascript
Javascript("""
function repeter(){
IPython.notebook.kernel.execute("kernelsurvivor = 1");
}
// execute a = 1 en python toutes les 8 minutes pendant 2h
let timerId = setInterval(() => repeter(), 4800);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); }, 7200000);

// Supprimer la taille limite pour la sortie d'une cellule
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};
IPython.notebook.kernel.execute("url = '" + window.location + "'");

// Exécuter toutes les cellule du notebook
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            
                
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');

        }
    );""")

<IPython.core.display.Javascript object>

In [40]:
from ipywidgets import Button, HBox, Layout, VBox,Label, HTML,FileUpload
import os
import time


download_bou = Button(
    description='Atrium',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Atrium',
    icon='download' # (FontAwesome names without the `fa-` prefix)
)


upload_bou = FileUpload(
    description='zip',
    disabled=False,
    accept=".zip",
    multiple=False,
    tooltip='zip',
    icon='upload' # (FontAwesome names without the `fa-` prefix)

)

message = Label(
    value = "")


def zipToServer(b):
    path = os.getcwd()
    os.chdir("/home/jovyan/")
    message.value = "decompression du fichier de sauvegarde"
    v = upload_bou.value
    
    with open(next(iter(upload_bou.value)), "w+b") as i:
        i.write(upload_bou.data[0])
    a = os.popen("""unzip -o {}""".format(next(iter(upload_bou.value))))
    m = ""
    for ligne in a:
        m += str(ligne)
    j = open("log_cadaver.txt","a")
    j.write("\n\n")
    j.write(m)
    j.close()
    os.chdir(path)
    message.value = "operation terminee avec succès"

    
def saveToAtrium(b):
    Javascript("""jupyter.actions.call('jupyter-notebook:save-notebook');""")
    path = os.getcwd()
    os.chdir("/home/jovyan/")
    t = time.localtime()
    message.value = "création du fichier de sauvegarde"
    a = os.popen("""zip -r nsipremiere{:4d}{:02d}{:02d}{:02d}{:02d}{:02d}.zip Perso""".format(t.tm_year,t.tm_mon,t.tm_mday,t.tm_hour,t.tm_min,t.tm_sec))
    m = ""
    for ligne in a:
        m += str(ligne)
    j = open("log_cadaver.txt","a")
    j.write("\n\n")
    j.write(m)
    j.close()
    
    j = open(".netrc","r")
    data = j.readlines()
    j.close()
    data = data[1]
    login = data.split(" ")[1].strip()
    
    message.value = "upload du fichier de sauvegarde vers atrium"
    j = open("datadav","w")
    j.write("cd {}\ncd document_library\nput /home/jovyan/nsipremiere{:4d}{:02d}{:02d}{:02d}{:02d}{:02d}.zip\nexit".format(login,t.tm_year,t.tm_mon,t.tm_mday,t.tm_hour,t.tm_min,t.tm_sec))
    j.close()
    a = os.popen("""cadaver -r datadav https://www.atrium-sud.fr/webdav""")
    m = ""
    for ligne in a:
        m += str(ligne)
    j = open("log_cadaver.txt","a")
    j.write("\n\n")
    j.write(m)
    j.close()
    if "succeeded" in m:
        message.value = "operation terminée avec succès"
    else:
        message.value = m
    os.chdir(path)


download_bou.on_click(saveToAtrium)
upload_bou.observe(zipToServer, "data")
display(HBox([download_bou,upload_bou]))
display(message)

Label(value='')